In [1]:
'''
git add .
git commit -m "update recorded" 
git push
'''

'\ngit add .\ngit commit -m "update recorded" \ngit push\n'

In [2]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import torch


from stable_baselines3.common.torch_layers import MlpExtractor      # for paramete of model
policy_kwargs = dict(
    net_arch=[dict(pi=[128, 64], vf=[128, 64])],  # policy and value net
)


# Constants representing the state of each cell
UNEXPLORED = -2
OBSTACLE = -1
SAFE = 0

# Initialize the global grid status as unexplored
global_grid = np.full((10, 10), UNEXPLORED)

def drone_scan(drone_pos, scan_range, actual_env):
    """
    Perform a local scan around a drone position.
    """
    half_range = scan_range // 2
    local_info = np.full((scan_range, scan_range), UNEXPLORED)

    grid_h, grid_w = actual_env.shape  # ✅ 动态获取地图大小！

    for i in range(scan_range):
        for j in range(scan_range):
            global_x = drone_pos[0] - half_range + i
            global_y = drone_pos[1] - half_range + j

            # ✅ 用实际地图的大小判断边界
            if 0 <= global_x < grid_h and 0 <= global_y < grid_w:
                local_info[i, j] = actual_env[global_x, global_y]

    return local_info, (drone_pos[0] - half_range, drone_pos[1] - half_range)


def stitch_information(global_grid, local_info, top_left):
    """
    Merge local drone scans into the global coverage grid.

    Args:
        global_grid (np.array): Current global grid state.
        local_info (np.array): Local scan result from a drone.
        top_left (tuple): Top-left coordinate of the local scan in the global grid.

    Returns:
        np.array: Updated global grid.
    """
    x_offset, y_offset = top_left
    grid_h, grid_w = global_grid.shape  # ✅ 动态获取地图大小！

    for i in range(local_info.shape[0]):
        for j in range(local_info.shape[1]):
            x, y = x_offset + i, y_offset + j
            if 0 <= x < grid_h and 0 <= y < grid_w:
                if global_grid[x, y] == UNEXPLORED:
                    global_grid[x, y] = local_info[i, j]
                elif global_grid[x, y] != local_info[i, j]:
                    if local_info[i, j] == SAFE:
                        global_grid[x, y] = SAFE
    return global_grid


In [3]:
# Example actual environment (randomly generated obstacles and safe zones when searching)
actual_env = np.random.choice([OBSTACLE, SAFE], size=(10, 10), p=[0.2, 0.8])

Training to put drones

In [4]:
from gymnasium import Env, spaces
import numpy as np

class DronePlacementEnv(Env):
    def __init__(self):
        super().__init__()
        self.grid_size = 10
        self.max_steps = 50
        self.current_step = 0

        # 10 x 10 grid × 2 drone types = 200 possible actions
        self.action_space = spaces.Discrete(200)
        self.observation_space = spaces.Box(low=UNEXPLORED, high=SAFE,shape=(self.grid_size, self.grid_size), dtype=np.int32)

        self.reset()

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.actual_env = np.random.choice([OBSTACLE, SAFE], size=(self.grid_size, self.grid_size), p=[0.2, 0.8]).astype(np.int32)

        self.global_grid = np.full((self.grid_size, self.grid_size), UNEXPLORED, dtype=np.int32)
        self.current_step = 0

        # ✅ 引导探索：预先把边缘随机几个点设为 SAFE
        edge_indices = list(range(self.grid_size))
        edge_cells = []

        # top and bottom rows
        for j in edge_indices:
            edge_cells.append((0, j))         # top row
            edge_cells.append((self.grid_size - 1, j))  # bottom row

        # left and right columns（去掉 corners 避免重复）
        for i in edge_indices[1:-1]:
            edge_cells.append((i, 0))         # left col
            edge_cells.append((i, self.grid_size - 1))  # right col

        # 随机选 10 个边缘格子填入已知 SAFE
        np.random.shuffle(edge_cells)
        for (x, y) in edge_cells[:10]:
            self.global_grid[x, y] = self.actual_env[x, y]

        # 明确暴露底部最多 3 行（不超过 grid 边界）
        for row in range(self.grid_size - 3, self.grid_size):
            self.global_grid[row, :] = self.actual_env[row, :]


        return self.global_grid.copy(), {}


    def step(self, action):
        self.current_step += 1
        action = int(min(action, self.grid_size * self.grid_size * 2 - 1))

        action_per_row = self.grid_size * 2  # 每行 2 个动作（3x3 + 5x5）

        x = action // action_per_row
        y = (action % action_per_row) // 2
        drone_type = action % 2

        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            raise ValueError(f"Decoded position ({x}, {y}) is out of bounds!")

        scan_range = 3 if drone_type == 0 else 5

        local_info, top_left = drone_scan((x, y), scan_range, self.actual_env)
        prev_unexplored = np.sum(self.global_grid == UNEXPLORED)
        self.global_grid = stitch_information(self.global_grid, local_info, top_left)
        new_unexplored = np.sum(self.global_grid == UNEXPLORED)

        # reward = new coverage - drone penalty - large drone penalty
        reward = float(prev_unexplored - new_unexplored - 0.2 - 0.2 * drone_type)

        # 边缘探索奖励：如果 drone 靠近边缘，就给鼓励
        if x <= 1 or x >= 8 or y <= 1 or y >= 8:
            reward += 0.3

        terminated = bool(new_unexplored == 0)
        truncated = bool(self.current_step >= self.max_steps)

        # 奖励完成地图探索
        if terminated:
            reward += 10  # 成功覆盖全图奖励
        elif truncated and np.sum(self.global_grid == UNEXPLORED) > 0:
            reward -= 5   # 达到最大步数却没扫完，惩罚


        return self.global_grid.copy(), reward, terminated, truncated, {}

    def render(self):
        print(self.global_grid)


In [5]:
# train a small grid 6*6, then the transfer to 10*10 grid

class SmallDroneEnv(DronePlacementEnv):
    def __init__(self):
        super().__init__()
        self.grid_size = 6
        self.max_steps = 8  # 适合小地图的 max steps
        self.action_space = spaces.Discrete(self.grid_size * self.grid_size * 2)
        self.observation_space = spaces.Box(low=UNEXPLORED, high=SAFE, shape=(6, 6), dtype=np.int32)

In [6]:
def train_on_small_map():
    env = SmallDroneEnv()
    check_env(env)
    model = PPO("MlpPolicy", env, verbose=2)
    model.learn(total_timesteps=50000)
    model.save("ppo_small_model")

    torch.save(model.policy.state_dict(), "small_policy_weights.pt")

    return model
'''
def train_with_ppo():
    env = DronePlacementEnv()
    check_env(env)
    #model = PPO("MlpPolicy", env)  # only for first time
    model = PPO.load("ppo_small_model", env=env, verbose=1)
    model.learn(total_timesteps=1e5)
    model.save("ppo_large_model")
    return model
'''
def train_with_ppo():
    env = DronePlacementEnv()
    check_env(env)
    
    # 与小地图保持一致的网络结构
    policy_kwargs = dict(net_arch=[dict(pi=[128, 64], vf=[128, 64])])
    model = PPO("MlpPolicy", env, verbose=1, policy_kwargs=policy_kwargs)

    # ✅ 加载小模型参数到新模型中（注意 strict=False）
    import torch
    model.policy.load_state_dict(torch.load("small_policy_weights.pt"), strict=False)

    model.learn(total_timesteps=10000)
    model.save("ppo_large_model")
    return model


In [7]:
def run_trained_model():
    model = PPO.load("ppo_large_model")
    env = DronePlacementEnv()
    obs, _ = env.reset()

    print("=== DRONE DEPLOYMENT SEQUENCE ===")
    for step in range(env.max_steps):
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)

        x = action // 20
        y = (action % 20) // 2
        drone_type = "3x3" if action % 2 == 0 else "5x5"
        print(f"Step {step}: Placed {drone_type} drone at ({x}, {y}), reward: {reward:.2f}")

        env.render()

        if terminated:
            print("🎉 Mission Complete: All cells explored!")
            break
        if truncated:
            print("⚠️ Max steps reached.")
            break

In [ ]:
if __name__ == "__main__":
    train_on_small_map()
    train_with_ppo()
    run_trained_model()

d:\anaconda3\envs\torchsafe\lib\site-packages\stable_baselines3\common\env_checker.py:272: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\anaconda3\envs\torchsafe\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.18     |
|    ep_rew_mean     | 17.3     |
| time/              |          |
|    fps             | 389      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------


In [ ]:
model = PPO.load("ppo_large_model")

env = DronePlacementEnv()
obs, _ = env.reset()

for step in range(env.max_steps):  # 最多放 max_steps 架 drone
    action, _ = model.predict(obs)

    # 解码动作
    x = action // 20
    y = (action % 20) // 2
    drone_type = action % 2
    scan_range = 3 if drone_type == 0 else 5
    pos = (x, y)

    # 模拟扫描 + 拼接
    local_info, top_left = drone_scan(pos, scan_range, env.actual_env)
    global_grid = stitch_information(env.global_grid, local_info, top_left)

    # 真正执行一步
    obs, reward, terminated, truncated, _ = env.step(action)

    print(f"Step {step}: Placed {scan_range}x{scan_range} drone at {pos}, reward = {reward:.2f}")
    env.render()

    if terminated:
        print("🎉 部署完成：所有格子已覆盖！")
        break
    if truncated:
        print("⚠️ 达到最大步数")
        break

print("Final Global Grid after Stitching:")
print(env.global_grid)
